# Notebook that does the processing of foraging session trajectories
'''Documentation about the task here'''

### Imports  
All functions are in behavior_analysis_functions.py

In [1]:
import os
import gc
import glob
import time
from behavior_analysis_functions import list_sessions_analyzed, \
                                    process_session, html_mouse, load_data, MOUSE_STRING




### Get mice data
Define the folder where your MOUXXX folder are, generate a list of MOUXXX folders and for each mice a list of sessions

In [ ]:
# the path_to_data_folder is the path of the folder where you store your different mice.

# Windows:
# path_to_data_folder='C:'+os.sep+'Users'+os.sep+'MORVAN'+os.sep+'Documents'+os.sep+'patchouris'+os.sep+'Sample_Data'+os.sep
# Linux:
# path_to_data_folder '/home/david/Documents/Code/ForagingProject/Patchouris/patchouris/Sample_Data/'
# MacOs
# path_to_data_folder = '/Users/davidrobbe/Documents/Science/Data/ForagingMice/'


path_to_data_folder='/LocalData/ForagingMice/JAK2Data'
pattern_of_MOU_Folders = os.path.join(path_to_data_folder, "MOU*")
#path_to_data_folder = '/Users/davidrobbe/Documents/Science/Data/ForagingMice/'

# List all mice in the data folder (If you want to process all the mice in your data folder),
mice_list = [os.path.basename(path) for path in glob.glob(pattern_of_MOU_Folders)]
mice_list=sorted(mice_list)
print(f'Found {len(mice_list)} {"mice" if len(mice_list) > 1 else "mouse"} in the data folder:')


#If you want to process a subset of mice uncomment the line below and comment the 2 lines above

#mice_list: list[str] = ["MOU2334"]  # For processing  a single mice. Equivalent to mice_list = ["MOU2334"] but more correct as it forces to create a list of string
#mice_list=['MOU2329', 'MOU2330', 'MOU2331', 'MOU2332', 'MOU2333', 'MOU2334']
#print(f'Found {len(mice_list)} {"mice" if len(mice_list) > 1 else "mouse"} in the data folder:')

session_list = {}
for mouse in mice_list:
    mouse_folder = os.path.join(path_to_data_folder,mouse)
    session_list[mouse] = sorted([name for name in os.listdir(mouse_folder)
                           if os.path.isdir(os.path.join(mouse_folder, name))
                           and name.startswith('MOU')])
    nb_sessions = len(session_list[mouse])
    print(f'Hello, I\'m {mouse}! I have foraged for {nb_sessions} sessions:')
    print(session_list[mouse], '\n')






## Process all the sessions of the mice in the mouse_list
### the key option here is wether to force processing or not (in case the data have already been processed)
#### for this the variable process should be set as true (to force) or false if the sessiobn has already been processed (if this is the case the name of the session has been saved in the  ListSessionsAnalyzed.txt file

In [ ]:
for mouse in mouse_list:
    print(f"Processing {mouse}")

    # Define and create analysis folder if it doesn't exist
    analysis_folder: str = path_to_data_folder + os.sep + mouse + os.sep + "Analysis"
    if not os.path.exists(analysis_folder):
        os.mkdir(analysis_folder)

    # Check and get list of analyzed sessions for this mouse
    txt: str = analysis_folder + os.sep + "ListSessionsAnalyzed.txt"
    listSessionAnalyzed: list[str] = list_sessions_analyzed(txt)

    # Process each session of the mouse if it is not in the list of analyzed 
    # sessions and write the session name in the list of analyzed sessions.
    # process_session() analyzes the session and saves the results
    # and figure in the Analysis folder.
    for i, session in enumerate(session_list[mouse]):
        print(f"{mouse}, session {i + 1} out of {len(session_list[mouse])}", end="")
        start_time: float = time.time()
        process: bool = False if session_list[mouse][i] in listSessionAnalyzed else True
        #process: bool = True  # Forcing processing of all sessions
        process_session(path_to_data_folder+mouse, session, process=process)
        if process:
            with open(txt, "a", encoding="utf-8") as f:
                # write the session name in the list of analyzed sessions if it is not already there
                if session not in listSessionAnalyzed:
                    f.write(session + "\n")
        gc.collect()
        print(f" processed in {time.time() - start_time:.1f} seconds")

    # Create html file with all sessions for this mouse
    html_mouse(path_to_data_folder, mouse)

### Process a defined subset of sessions of a single mice in the mouse_list

In [ ]:
# same of code as above if you want to process a single session for a single mouse (obviously yu can process two or three)
# mouse="MOU2334"
# session_list=['MOU2334_20240614-1015']
mouse="MOU4175"
session_list=['MOU4175_20230712-1809']

print(f"Processing {mouse}")

# Define and create analysis folder if it doesn't exist
analysis_folder: str = path_to_data_folder + os.sep + mouse + os.sep + "Analysis"
if not os.path.exists(analysis_folder):
    os.mkdir(analysis_folder)



# Process each session of the mouse if it is not in the list of analyzed 
# sessions and write the session name in the list of analyzed sessions.
# process_session() analyzes the session and saves the results
# and figure in the Analysis folder.
for i, session in enumerate(session_list):
    print(f"{mouse}, session {i + 1} out of {len(session_list)}", end="")
    start_time: float = time.time()
    #process: bool = False if session_list[mouse][i] in listSessionAnalyzed else True
    process: bool = True  # Forcing processing of all sessions
    list_epochs=process_session(path_to_data_folder+mouse, session, process=process)
    # if process:
    #     with open(txt, "a", encoding="utf-8") as f:
    #         # write the session name in the list of analyzed sessions if it is not already there
    #         if session not in listSessionAnalyzed:
    #             f.write(session + "\n")
    gc.collect()
    print(f" processed in {time.time() - start_time:.1f} seconds")

# Create html file with all sessions for this mouse
#html_mouse(path_to_data_folder, mouse)